### <center> Modelo SIR para la Poblacion del Ecuador  </center>

Es un modelo matemático que permite predecir el comportamiento de una enfermedad infecciosa, a partir de ciertas condiciones iniciales y clasifica una población en tres grupos distintos:

-  Suceptible: Número de personas que son propensas a la enfermedad
-  Infectado: Número de personas que tienen la enfermedad y que pueden infectar a la gente susceptible
-  Recuperado: Númeto de persoans que no pueden contraer la enfermedad, porque se han recuperado completamente, o porque son inmunes

In [7]:
#IMPORTAMOS LAS LIBRERIAS NECESARIAS

import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import altair as alt
import matplotlib.pyplot as plt
import scipy.integrate as spi
from scipy.optimize import minimize
from scipy.integrate import solve_ivp #permiten resolver sistemas de ecuaciones, integrarlas y graficarlas

N     = 17268000 #Número de población
I0    = 10000  #Número inicial de infectados
R0    = 0 #Número inicial de recuperados
S0    = N - I0 - R0
beta  = 0.3 #Tasa de transmisión
#Tasa de recuperación teniendo en cuenta que una persona puede recuperarse en 15 días, es decir 1/15
gamma = 1/15
t     = np.linspace(0, 365, 365)

def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

y0 = S0, I0, R0
ret = spi.odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

#dataframe que nos explica como va variado el SIR 
#normalizamos de 0 a 1 ya que estan divididos para el número de población
df = pd.DataFrame({'time':t.astype('int'),'susceptible':S/N,'infected':I/N,'recovered':R/N})
df.head(5)

,time,susceptible,infected,recovered
0,0,0.999421,0.000579,0.000000
1,1,0.999225,0.000732,0.000044
2,2,0.998977,0.000924,0.000099
3,3,0.998664,0.001167,0.000168
4,4,0.998269,0.001475,0.000256


In [8]:
alt.Chart(df.melt('time')).mark_line().encode(
    x='time',
    y=alt.Y('value'),
    color='variable'
).properties(
    title='SIR model for Ecuador Example'
).interactive()

alt.Chart(...)

In [9]:
# Simulación del modelo SIR para COVID-19

I0=10
R0=0
S0 = 100000
t = 365
y0 = S0,I0,R0

def filter_country(df,country,start_date):
    country_df = df[df['Country/Region'] == country]
    return country_df.iloc[0].loc[start_date:]

def load_data(path_confirmed,path_recovered,country,date):
    df_confirmed = filter_country(pd.read_csv(path_confirmed),country,date)
    df_recovered = filter_country(pd.read_csv(path_recovered),country,date)
    return df_confirmed,df_recovered

In [10]:
data_confirmed,data_recovered=load_data('time_series_covid19_confirmed_global.csv','time_series_covid19_recovered_global.csv','Ecuador','3/1/20')

In [11]:
def loss_confirmed_recovered(point, data, recovered):
    size = len(data)
    beta, gamma = point
    def SIR(t, y):
        S = y[0]
        I = y[1]
        R = y[2]
        return [-beta*S*I, beta*S*I-gamma*I, gamma*I]
    solution = solve_ivp(SIR, [0, size], [S0,I0,R0], t_eval=np.arange(0, size, 1), vectorized=True)
    l1 = np.sqrt(np.mean((solution.y[1] - data)**2))
    l2 = np.sqrt(np.mean((solution.y[2] - recovered)**2))
    alpha = 0.1
    return alpha * l1 + (1 - alpha) * l2


def loss_confirmed(point, data):
    size = len(data)
    beta, gamma = point
    def SIR(t, y):
        S = y[0]
        I = y[1]
        R = y[2]
        return [-beta*S*I, beta*S*I-gamma*I, gamma*I]
    solution = solve_ivp(SIR, [0, size], [S0,I0,R0], t_eval=np.arange(0, size, 1), vectorized=True)
    return np.sqrt(np.mean((solution.y[1] - data)**2))

In [12]:
#Optimización 

optimal_cr = minimize(
            loss_confirmed_recovered,
            [0.001, 0.001],
            args=(data_confirmed,data_recovered),
            method='L-BFGS-B',
            bounds=[(0.00000001, 0.4), (0.00000001, 0.4)])

optimal_c = minimize(
            loss_confirmed,
            [0.001, 0.001],
            args=(data_confirmed),
            method='L-BFGS-B',
            bounds=[(0.00000001, 0.4), (0.00000001, 0.4)])

In [13]:
beta_cr,gamma_cr = optimal_cr.x
beta_c,gamma_c = optimal_c.x

# valores que obtuvimos con la primera función
print("VALORES DE LA PRIMERA FUNCION")

print("valor gamma: {}".format(gamma_cr))
print("valor beta: {}".format(beta_cr))

print("VALORES DE LA SEGUNDA FUNCION")

print("valor gamma: {}".format(gamma_c))
print("valor beta: {}".format(beta_c))

VALORES DE LA PRIMERA FUNCION
valor gamma: 0.011075261725669203
valor beta: 0.011083742218320881
VALORES DE LA SEGUNDA FUNCION
valor gamma: 4.531191647686112e-06
valor beta: 4.531191647686112e-06


In [14]:
#En este caso mostrarmos el R_0 con los valores de beta y gamma de la primera función
R_0= beta_cr/gamma_cr
print("Número de reproducción R_0: {}".format(R_0))

#R_0 con los valores de beta y gamma de la segunda función
R_0= beta_c/gamma_c
print("Número de reproducción R_0: {}".format(R_0))

Número de reproducción R_0: 1.0007657148753444
Número de reproducción R_0: 1.0


In [15]:
def extend_index(index, new_size):
        values = index.values
        current = datetime.strptime(index[-1], '%m/%d/%y')
        while len(values) < new_size:
            current = current + timedelta(days=1)
            values = np.append(values, datetime.strftime(current, '%m/%d/%y'))
        return values

def predict(beta, gamma, data):
        predict_range = t
        new_index = extend_index(data.index, predict_range)
        size = len(new_index)
        def SIR(t, y):
            S = y[0]
            I = y[1]
            R = y[2]
            return [-beta*S*I, beta*S*I-gamma*I, gamma*I]
        extended_actual = np.concatenate((data.values, [None] * (size - len(data.values))))
        return new_index, extended_actual, solve_ivp(SIR, [0, size], [S0,I0,R0], t_eval=np.arange(0, size, 1))

In [16]:
#predicciones para nuestros datos, teniendo en cuenta el beta y gamma. 
#A continuación realizamos dichas predicciones para la primera y segunda función
new_index, extended_actual, prediction_cr = predict(beta_cr, gamma_cr, data_confirmed)
new_index, extended_actual, prediction_c = predict(beta_c, gamma_c, data_confirmed)

In [17]:
#PRIMERA FUNCIONA DATAGRAMA
df_cr = pd.DataFrame(
            {'date':[i for i in range(0,len(new_index))],
            'suceptible': prediction_cr.y[0],
            'infected': prediction_cr.y[1],
            'recovered': prediction_cr.y[2]})

#SEGUNDA FUNCIONA DATAGRAMA
df_c = pd.DataFrame(
            {'date':[i for i in range(0,len(new_index))],
            'suceptible': prediction_c.y[0],
            'infected': prediction_c.y[1],
            'recovered': prediction_c.y[2]})

In [18]:
df_cr.head(5)

,date,suceptible,infected,recovered
0,0,1.000000e+05,10.000000,0.000000
1,1,-4.098492e-07,98917.574493,1092.425507
2,2,7.499685e-07,97828.080816,2181.919184
3,3,-1.064281e-06,96750.586995,3259.413006
4,4,-3.455347e-07,95684.960858,4325.039142


In [19]:
df_c.head(5)

,date,suceptible,infected,recovered
0,0,100000.000000,10.000000,0.000000
1,1,99994.268098,15.731844,0.000057
2,2,99985.248471,24.751382,0.000148
3,3,99971.080854,38.918857,0.000289
4,4,99948.767098,61.232389,0.000512


In [20]:
#GRAFICA DE LA PRIMERA FUNCION
alt.Chart(df_cr.melt('date')).mark_line().encode(
    x='date',
    y=alt.Y('value'),
    color='variable'
).properties(
    title='Modelo SIR para Ecuador con casos confirmados y recuperados'
).interactive()

alt.Chart(...)

In [21]:
#GRAFICA DE LA SEGUNDA FUNCION
alt.Chart(df_c.melt('date')).mark_line().encode(
    x='date',
    y=alt.Y('value'),
    color='variable'
).properties(
    title='Modelo SIR para Ecuador solo casos confirmados'
).interactive()

alt.Chart(...)